In [3]:
#Imports!:!:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR  # for building SVR model
import sklearn.metrics as sm
from gradients import compute_face_phi, dphidx, dphidy, init
import warnings
import matplotlib.cbook
import time
import sys

In [6]:
a = np.arange(6).reshape((3, 2))
a
a = a.reshape(-1,1)
a


array([[0],
       [1],
       [2],
       [3],
       [4],
       [5]])

get_ni_nj returns the wanted dimentions on the different cases. It could probably be decided based on the path names, but we use magic numbers. 

In [ ]:
def get_ni_nj(x:np.array, y:np.array):
    if max(y) == 1.:
        ni,nj = 170,194
        nu = 1. / 10000.
    else:
        nu = 1. / 10595.
        if max(x) > 8.:
            ni,nj = 162,162
        else:
            ni,nj = 402,162
    return nu,ni,nj  

this is a version that matches on path names:

In [1]:
def get_ni_nj(path:str):

    nu,ni,nj = 0,0,0
    
    if path == "small_wave/tec.dat" or path == "large_wave/tec_large.dat":
        nu,ni,nj = 0.0001,170,194
    if path == "one_hill/tec_OneHill.dat":
        nu,ni,nj = (1./10595.),162,162
    if path == "two_hills/tecTwoHills.dat":
        nu,ni,nj = (1./10595.),402,162

    return nu,ni,nj

In [2]:
def dat_to_variable_arrays(path:str):
    tec = np.genfromtxt(path, dtype=None, comments="%").transpose() #turn every column in the .dat file to a row in tec
    k_DNS = 0.5 * (tec[5] + tec[6] + tec[7])                        #Turbulent kinetic energy (?)

    nu,ni,nj = get_ni_nj(path)                      #get the dimentions on of the data, based on path name

    k_DNS = np.reshape(k_DNS, (nj, ni)).transpose() #k_DNS has the shape [ni,nj]
    tec = tec.reshape(10,nj,ni)                     #tec is reshaped so every variable has the shape [nj,ni]
    tec = np.transpose(tec,axes=[0,2,1])            #tec is transposed so that every variable has the shape [ni,nj]

    x       = tec[0]
    y       = tec[1]
    p       = tec[2]
    u       = tec[3]
    v       = tec[4]
    uu      = tec[5]
    vv      = tec[6]
    ww      = tec[7]
    uv      = tec[8]
    eps_DNS = tec[9]

    return x,y,p,u,v,uu,vv,ww,uv,eps_DNS,k_DNS,ni,nj